*My CPU was facing some issue with the full dataset while converting from dataframe. Hence, I took random sample of days and time from the full dataset and prepared the assignment.*

In [1]:
# Import required libraries
import re
import os
from datetime import datetime as dt
from collections import OrderedDict, Counter
import pandas as pd
from pandas.compat import StringIO
import numpy as np

# Define the input log file
file = 'data/nasa-http/NASA_access_log_Aug95.log'

# Create regular expression to parse the web log file
log_line_regex = re.compile(''.join([
    r'^(?P<host>[\S]+)\s-\s-\s', r'\[(?P<timestamp>.{26})\]',
    r'\s"(?P<request_method>[A-Z]{3,4})\s(?P<request_url>.{1,100})(\sHTTP/1.0")?',
    r'\s(?P<reply_code>[0-9]{3})\s(?P<reply_bytes>[0-9-]{1,20})$'
]))

In [2]:
# Create empty dataframe
nasa = pd.DataFrame([])
loglst = list()

# Set count to loop through lines in the file
cnt = 0
with open(file) as fl:
    for line in fl:
        m = log_line_regex.match(line)
        record = OrderedDict([
            (key, value) 
            for key, value in m.groupdict().items()
        ])
        
        # I commented this because pandas was having problen to convert a field of datetime.datetime and datetime.timezone
        # Since timezone is not required in the analysis, I am reading this as string and then converting to datetime
        #record['timestamp'] = datetime.strptime(
            #record['timestamp'], '%d/%b/%Y:%H:%M:%S %z')
        
        loglst.append(record)

In [3]:
# Convert the list of OrderedDict to a dataframe
col = Counter()
for k in loglst:
    col.update(k)

df = pd.DataFrame([k.values() for k in loglst], columns = col.keys())
    
df.head()

,host,timestamp,request_method,request_url,reply_code,reply_bytes
0,uplherc.upl.com,01/Aug/1995:00:00:08 -0400,GET,"/images/USA-logosmall.gif HTTP/1.0""",304,0
1,ix-esc-ca2-07.ix.netcom.com,01/Aug/1995:00:00:09 -0400,GET,"/images/launch-logo.gif HTTP/1.0""",200,1713
2,uplherc.upl.com,01/Aug/1995:00:00:10 -0400,GET,"/images/WORLD-logosmall.gif HTTP/1.0""",304,0
3,slppp6.intermind.net,01/Aug/1995:00:00:10 -0400,GET,"/history/skylab/skylab.html HTTP/1.0""",200,1687
4,piweba4y.prodigy.com,01/Aug/1995:00:00:10 -0400,GET,"/images/launchmedium.gif HTTP/1.0""",200,11853


In [4]:
# Create date time column from string
df['DateTime'] = df['timestamp'].str.split(' ').str[0]
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%d/%b/%Y:%H:%M:%S')

# Extract date from datetime
df['Date'] = df['DateTime'].dt.date

#df['Week'] = df['Date'].apply(lambda x: (x + pd.Timedelta(days=1)).week)

# Extract day names from datetime
df['DayName'] = df['DateTime'].dt.weekday_name

# Extract day of week from datetime
df['DayOfWeek'] = df['DateTime'].dt.dayofweek

# Extract hour from datetime
df['Hour'] = df['DateTime'].dt.hour.apply(pd.to_numeric, errors='coerce')

# Convert reply bytes to int
df['reply_bytes'] = df['reply_bytes'].apply(pd.to_numeric, errors='coerce')

# Add a dummy column for calculating sum later
df['Cnt'] = 1

# Add Index
#df['Idx'] = df.index

In [5]:
# Create an hour bin to map the hour in the request to the corresponding bin
bins = pd.DataFrame({'low':[0,4,7,10,13,16,19,22],
                  'high':[3,6,9,12,15,18,21,23],
                  'name':['00:00 to 03:00',
                          '03:00 to 06:00',
                          '06:00 to 09:00',
                          '09:00 to 12:00',
                          '12:00 to 15:00',
                          '15:00 to 18:00',
                          '18:00 to 21:00',
                          '21:00 to 24:00']})

# Create mapping function
def hourmap(x):
    for row in bins.itertuples():
        if row.low <= x <= row.high:
            return row.name

# Apply the mapping to the hour
df['Time of Day'] = df.Hour.map(hourmap)

df.head()

,host,timestamp,request_method,request_url,reply_code,reply_bytes,DateTime,Date,DayName,DayOfWeek,Hour,Cnt,Time of Day
0,uplherc.upl.com,01/Aug/1995:00:00:08 -0400,GET,"/images/USA-logosmall.gif HTTP/1.0""",304,0,1995-08-01 00:00:08,1995-08-01,Tuesday,1,0,1,00:00 to 03:00
1,ix-esc-ca2-07.ix.netcom.com,01/Aug/1995:00:00:09 -0400,GET,"/images/launch-logo.gif HTTP/1.0""",200,1713,1995-08-01 00:00:09,1995-08-01,Tuesday,1,0,1,00:00 to 03:00
2,uplherc.upl.com,01/Aug/1995:00:00:10 -0400,GET,"/images/WORLD-logosmall.gif HTTP/1.0""",304,0,1995-08-01 00:00:10,1995-08-01,Tuesday,1,0,1,00:00 to 03:00
3,slppp6.intermind.net,01/Aug/1995:00:00:10 -0400,GET,"/history/skylab/skylab.html HTTP/1.0""",200,1687,1995-08-01 00:00:10,1995-08-01,Tuesday,1,0,1,00:00 to 03:00
4,piweba4y.prodigy.com,01/Aug/1995:00:00:10 -0400,GET,"/images/launchmedium.gif HTTP/1.0""",200,11853,1995-08-01 00:00:10,1995-08-01,Tuesday,1,0,1,00:00 to 03:00


In [6]:
# Create dataframe for calculating number of requests
dfReqNo = df.groupby(['Date', 'DayName', 'Time of Day'])['Cnt'].agg('sum').reset_index(name='NoOfReq')

dfReqNo.head()

,Date,DayName,Time of Day,NoOfReq
0,1995-08-01,Tuesday,00:00 to 03:00,17
1,1995-08-03,Thursday,03:00 to 06:00,8
2,1995-08-08,Tuesday,15:00 to 18:00,8
3,1995-08-09,Wednesday,21:00 to 24:00,10
4,1995-08-10,Thursday,18:00 to 21:00,7


In [7]:
# For sorting, index the days with week starting on Sunday (as directed in the assignment)
sorter = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
sorterIndex = dict(zip(sorter,range(len(sorter))))
sorterIndex

{'Sunday': 0,
 'Monday': 1,
 'Tuesday': 2,
 'Wednesday': 3,
 'Thursday': 4,
 'Friday': 5,
 'Saturday': 6}

**a. Requests per Day of Week**

In [8]:
dfReqnumDay = dfReqNo.groupby('DayName').agg({'NoOfReq':[np.mean,np.min,np.max]})
dfReqnumDay.columns = ['Requests (Mean)','Requests (Min)','Requests (Max)']

# Apply weekday sort
dfReqnumDay['Day_id'] = dfReqnumDay.index
dfReqnumDay['Day_id'] = dfReqnumDay['Day_id'].map(sorterIndex)
dfReqnumDay.sort_values('Day_id', inplace=True)

# Select, dropping the weekday index
dfReqnumDay[dfReqnumDay.columns.difference(['Day_id'])]

,Requests (Max),Requests (Mean),Requests (Min)
DayName,,,
Sunday,5,3.666667,1
Monday,6,2.750000,1
Tuesday,17,6.600000,1
Wednesday,10,4.750000,2
Thursday,8,3.833333,1
Friday,5,2.500000,1
Saturday,7,3.666667,1


**b. Requests by Time of Day**

In [9]:
dfReqnumTime = dfReqNo.groupby('Time of Day').agg({'NoOfReq':[np.mean,np.min,np.max]})
dfReqnumTime.columns = ['Requests (Mean)','Requests (Min)','Requests (Max)']

dfReqnumTime

,Requests (Mean),Requests (Min),Requests (Max)
Time of Day,,,
00:00 to 03:00,4.333333,1,17
03:00 to 06:00,5.750000,3,8
06:00 to 09:00,2.200000,1,3
09:00 to 12:00,2.333333,1,5
12:00 to 15:00,3.000000,1,5
15:00 to 18:00,7.000000,6,8
18:00 to 21:00,3.750000,1,7
21:00 to 24:00,10.000000,10,10


**c. Data Transfer per Day of Week**

In [10]:
# Create dataframe for calculating data transfer volume
dfDataVol = df.groupby(['Date', 'DayName'])['reply_bytes'].agg('sum').reset_index(name='DataVolume')

dfDataVol.head()

,Date,DayName,DataVolume
0,1995-08-01,Tuesday,80511
1,1995-08-03,Thursday,121717
2,1995-08-08,Tuesday,12294
3,1995-08-09,Wednesday,134343
4,1995-08-10,Thursday,247286


In [11]:
dfDatavolDay = dfDataVol.groupby('DayName').agg({'DataVolume':[np.mean,np.min,np.max]})
dfDatavolDay.columns = ['Mean Bandwidth (MB)','Min Bandwidth (MB)','Max Bandwidth (MB)']

# Apply weekday sort
dfDatavolDay['Day_id'] = dfDatavolDay.index
dfDatavolDay['Day_id'] = dfDatavolDay['Day_id'].map(sorterIndex)
dfDatavolDay.sort_values('Day_id', inplace=True)

# Select, dropping the weekday index
dfDatavolDay[dfDatavolDay.columns.difference(['Day_id'])]

,Max Bandwidth (MB),Mean Bandwidth (MB),Min Bandwidth (MB)
DayName,,,
Sunday,141303,90638.333333,14176
Monday,140669,53171.333333,4556
Tuesday,307257,84451.000000,669
Wednesday,134343,51957.500000,3250
Thursday,247286,93783.800000,1149
Friday,84874,45693.333333,9881
Saturday,56123,24496.666667,1932


**End of Code**